<a href="https://colab.research.google.com/github/AnSe2002/Storage_processing_big_data/blob/main/%D0%A1%D0%B5%D1%80%D0%B3%D0%B5%D0%B5%D0%B2%D0%B0_%D0%90%D0%BD%D0%B0%D1%81%D1%82%D0%B0%D1%81%D0%B8%D1%8F_lab_6_Spark_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 style="text-align:center;font-size:200%;;">Выполните разведочный анализ датасета с определением: </h2>
<h3  style="text-align:center;">

<span class="label label-success">- типов признаков в датасете;</span>

<span class="label label-success">- пропущенных значений и их устранением;</span>

<span class="label label-success">- выбросов и их устранением;</span>

<span class="label label-success">- расчетом статистических показателей признаков (средних, квартилей и т.д.);</span>

<span class="label label-success">- визуализацией распределения наиболее важных признаков;</span>

<span class="label label-success">- корреляций между признаками;</span>

<span class="label label-success">- реализовать несколько UDF для работы с данными.</span></h3>

# **1. Инициализация платформы Spark и загрузка данных в фрейм данных Spark** <a class="anchor" id="1"></a>


Источник данных: https://www.kaggle.com/datasets/willianoliveiragibin/uk-property-price-data-1995-2023-04/data

In [2]:
# Импорт модулей, не связанныех с PySpark.
import os
import sys
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
# Это помогает автоматически распечатывать элементы без явного использования «печати».
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [3]:
! pip install kaggle

In [4]:
!pip install opendatasets

Выгрузила данные про продажи недвижимости в Англии и Уэльсе, полученные из земельного реестра правительства Великобритании HM. Он предлагает ценную информацию о сделках с недвижимостью, включая цены продажи, местоположение и типы продаваемых объектов. Этот набор данных особенно полезен аналитикам, исследователям и компаниям, желающим понять рыночные тенденции, оценки недвижимости и инвестиционные возможности в секторе недвижимости Англии и Уэльса.

In [5]:
# Выгрузить данные из Kaggle

import opendatasets as od
import pandas

od.download(
    "https://www.kaggle.com/datasets/willianoliveiragibin/uk-property-price-data-1995-2023-04/data?select=202304.csv")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: anastasiasergeevaai
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/willianoliveiragibin/uk-property-price-data-1995-2023-04


100%|██████████| 1.36G/1.36G [01:06<00:00, 22.1MB/s]


In [6]:
! pip install pyspark

In [7]:
# Импорт модулей, связанных с PySpark.
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct
from pyspark.sql.types import *

MAX_MEMORY = '15G'
# Инициализировать сеанс Spark.
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Pyspark guide") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

# Выгрузим данные в формате CSV
spark = init_spark()
df = spark.read.csv("uk-property-price-data-1995-2023-04/202304.csv")
print('Data frame type: ' + str(type(df)))

Data frame type: <class 'pyspark.sql.dataframe.DataFrame'>


# **2. Обзор набора данных** <a class="anchor" id="2"></a>

### Схема, столбцы и типы данных набора: все данные строковые.

In [ ]:
print('Обзор данных')
df.printSchema()
print('Обзор столбцов')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Обзор данных
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)

Обзор столбцов


,Column Name,Data type
0,_c0,string
1,_c1,string
2,_c2,string
3,_c3,string
4,_c4,string
5,_c5,string
6,_c6,string
7,_c7,string
8,_c8,string
9,_c9,string


Видно, что названия столбцов отображены неверно, названия были взяты из данных с сайта Kaggle.

Postcode: почтовый индекс, по которому расположена недвижимость.

<span class="label label-success">PAON (основное адресное название объекта): обычно номер или название дома.</span>

<span class="label label-success">SAON (вторичное адресное название объекта): дополнительная информация, если здание разделено на квартиры или субподстройки.</span>

<span class="label label-success">Street: название улицы, на которой расположена недвижимость.</span>

<span class="label label-success">Locality: Дополнительная информация о населенном пункте.</span>

<span class="label label-success">Town/City: населенный пункт, в котором находится недвижимость.</span>

<span class="label label-success">District: Округ, в котором находится недвижимость.</span>

<span class="label label-success">County: округ, в котором расположена недвижимость.</span>

<span class="label label-success">Price Paid: цена, по которой была продана недвижимость.</span>

<span class="label label-success">Property_Type: тип собственности.

In [8]:
Data_list = ["Transaction_unique_identifier", "Price Paid", "Date_of_Transfer", "Postcode", "Property_Type", "Old/New",
             "Duration", "PAON", "SAON", "Street", "Locality", "Town/City", "District", "County", "PPDCategory_Type",
             "Record_Status - monthly_file_only"]

new_df = df.toDF(*Data_list)
new_df.show()

+-----------------------------+----------+----------------+--------+-------------+-------+--------+--------------+----+-----------------+-------------+-------------+-------------------+---------------+----------------+---------------------------------+
|Transaction_unique_identifier|Price Paid|Date_of_Transfer|Postcode|Property_Type|Old/New|Duration|          PAON|SAON|           Street|     Locality|    Town/City|           District|         County|PPDCategory_Type|Record_Status - monthly_file_only|
+-----------------------------+----------+----------------+--------+-------------+-------+--------+--------------+----+-----------------+-------------+-------------+-------------------+---------------+----------------+---------------------------------+
|         {F887F88E-7D15-44...|     70000|1995-07-07 00:00|MK15 9HP|            D|      N|       F|            31|NULL|    ALDRICH DRIVE|       WILLEN|MILTON KEYNES|      MILTON KEYNES|  MILTON KEYNES|               A|                       

In [9]:
from pyspark.sql.functions import col, to_timestamp
new_df = new_df.withColumn("Date_of_Transfer", to_timestamp(col("Date_of_Transfer"), "yyyy-MM-dd HH:mm"))
new_df = new_df.withColumn("Price Paid", col("Price Paid").cast("integer"))

In [ ]:
new_df.describe().show()

+-------+-----------------------------+------------------+----------------+--------+-------------+--------+--------+--------------------+--------------------+------------+-----------+--------------+---------+--------+----------------+---------------------------------+
|summary|Transaction_unique_identifier|        Price Paid|Date_of_Transfer|Postcode|Property_Type| Old/New|Duration|                PAON|                SAON|      Street|   Locality|     Town/City| District|  County|PPDCategory_Type|Record_Status - monthly_file_only|
+-------+-----------------------------+------------------+----------------+--------+-------------+--------+--------+--------------------+--------------------+------------+-----------+--------------+---------+--------+----------------+---------------------------------+
|  count|                     28276228|          28276228|        28276228|28230259|     28276228|28276228|28276228|            28272039|             3325860|    27828472|   18190377|      2827

В наборе данных содержится 28.276.228 записей о трансакциях с недвижимостью.

Средняя цена, уплаченная за недвижимость, составляет примерно 218.130,49. Это указывает на то, что большинство сделок происходит по ценам, близким к этому значению, хотя могут быть значительные отклонения (стандартное отклонение составляет около  868.852,45).

Минимальная цена, зафиксированная в наборе данных, составляет 1, что может указывать на случайные ошибки.

Максимальная цена составляет 9.999.999, что может означать высокие сделки с элитной недвижимостью.

Даты варьируются с 1 января 1995 года до 27 апреля 2023 года.

Поле Record_Status показывает, что все записи имеют статус "A". Это может обозначать, что все данные актуальны и доступны для анализа.

Далее из таблицы были удалены дубли по уникальному идентификатору транзакции.

In [10]:
df_updated = new_df.dropDuplicates(["Transaction_unique_identifier"])

df_updated.show()

+-----------------------------+----------+-------------------+--------+-------------+-------+--------+-------------------+-------+-----------------+-------------+---------------+--------------------+----------------+----------------+---------------------------------+
|Transaction_unique_identifier|Price Paid|   Date_of_Transfer|Postcode|Property_Type|Old/New|Duration|               PAON|   SAON|           Street|     Locality|      Town/City|            District|          County|PPDCategory_Type|Record_Status - monthly_file_only|
+-----------------------------+----------+-------------------+--------+-------------+-------+--------+-------------------+-------+-----------------+-------------+---------------+--------------------+----------------+----------------+---------------------------------+
|         {00003644-982F-49...|    275000|2014-06-27 00:00:00| BB7 1ER|            D|      N|       L|                 36|   NULL|COPPERFIELD CLOSE|         NULL|      CLITHEROE|       RIBBLE VALL

In [ ]:
print('Обзор данных')
df_updated.printSchema()

Обзор данных
root
 |-- Transaction_unique_identifier: string (nullable = true)
 |-- Price Paid: integer (nullable = true)
 |-- Date_of_Transfer: timestamp (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Property_Type: string (nullable = true)
 |-- Old/New: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- PAON: string (nullable = true)
 |-- SAON: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Town/City: string (nullable = true)
 |-- District: string (nullable = true)
 |-- County: string (nullable = true)
 |-- PPDCategory_Type: string (nullable = true)
 |-- Record_Status - monthly_file_only: string (nullable = true)



# **3. Обнаружение пропущенных значений и аномальных нулей.** <a class="anchor" id="3"></a>

После предварительного просмотра столбцов первое, что должны проверить, — это наличие в наборе данных какого-либо пропущенного значения.
– Для строковых столбцов  проверяем наличие `None` и `null`.
- Для числовых столбцов проверяем наличие нулей и `NaN`.

In [ ]:
string_columns = ["Transaction_unique_identifier", "Postcode", "Property_Type", "Old/New",
                  "Duration", "PAON", "SAON", "Street", "Locality", "Town/City", "District", "County", "PPDCategory_Type",
                  "Record_Status - monthly_file_only"]
numeric_columns = ["Price Paid", "Date_of_Transfer"]
date_columms1 =  ["Date_of_Transfer"]
missing_values = {}
for index, column in enumerate(df_updated.columns):
    if column in string_columns:    # проверить столбцы строк со значениями None и Null
        missing_count = df_updated.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
        missing_values.update({column:missing_count})
    if column in numeric_columns:  # check zeroes, None, NaN
        missing_count = df_updated.where(col(column).isin([0,None,np.nan])).count()
        missing_values.update({column:missing_count})
    if column in date_columms1:  # check zeroes, None, NaN
        missing_count1 = df_updated.where(col(column).isNull()).count()
        missing_values.update({column:missing_count})
missing_df = pd.DataFrame.from_dict([missing_values])
missing_df

,Transaction_unique_identifier,Price Paid,Date_of_Transfer,Postcode,Property_Type,Old/New,Duration,PAON,SAON,Street,Locality,Town/City,District,County,PPDCategory_Type,Record_Status - monthly_file_only
0,0,0,0,45969,0,0,0,4189,24950368,447756,10085851,0,0,0,0,0


У нас отсутствуют некоторые данные по расположению домов, а именно PAON, Street, SAON, Locality. Почтовые индексы Великобритании не всегда соответствуют конкретным улицам, поскольку они предназначены для обозначения более широких районов. Одна улица может иметь несколько почтовых индексов, в зависимости от ее протяженности и расположения. При этом поля Locality и SAON необязательны для запонения, т.к. отвечают за дополнительную информацию, которая необязательно могла быть указана. Поле с почтовым индексом несет в себе не так много информации для анализа, как адрес улицы.

In [ ]:
# Фильтрация DataFrame
filtered_df = df_updated.filter(
    (col("Postcode").isNotNull()) & (col("Postcode") != "") &  # Postcode заполнен
    (col("Street").isNull() | (col("Street") == ""))        # Street пустой
)

filtered_df.show()

print("Количество строк, в которых Postcode заполнен, а Street пуст:", filtered_df.count())

+-----------------------------+----------+----------------+--------+-------------+-------+--------+--------------------+------------+------+----------------+--------------+--------------------+--------------------+----------------+---------------------------------+
|Transaction_unique_identifier|Price Paid|Date_of_Transfer|Postcode|Property_Type|Old/New|Duration|                PAON|        SAON|Street|        Locality|     Town/City|            District|              County|PPDCategory_Type|Record_Status - monthly_file_only|
+-----------------------------+----------+----------------+--------+-------------+-------+--------+--------------------+------------+------+----------------+--------------+--------------------+--------------------+----------------+---------------------------------+
|         {0004509B-7312-42...|    245000|2002-05-24 00:00| HR3 6QN|            D|      N|       F| LITTLE WOONTON FARM|     HAYWAIN|  NULL|         WOONTON|      HEREFORD|       HEREFORDSHIRE|       HE

Было принято решение сделать анализ без учета улиц. Был выполнен срез данных и убраны неключевые столбцы с улицей, почтовым индексом, домом и дополнительной информацией о продаже.

In [11]:
df_updated_1 = df_updated.drop("Postcode", "SAON", "PAON", "Street", "Locality", "Record_Status - monthly_file_only", "PPDCategory_Type")
df_updated_1.show()

+-----------------------------+----------+-------------------+-------------+-------+--------+-----------------+--------------------+------------------+
|Transaction_unique_identifier|Price Paid|   Date_of_Transfer|Property_Type|Old/New|Duration|        Town/City|            District|            County|
+-----------------------------+----------+-------------------+-------------+-------+--------+-----------------+--------------------+------------------+
|         {0000012D-3A97-4F...|     52500|1997-06-27 00:00:00|            S|      N|       F|          BRISTOL|      NORTH SOMERSET|    NORTH SOMERSET|
|         {00003644-982F-49...|    275000|2014-06-27 00:00:00|            D|      N|       L|        CLITHEROE|       RIBBLE VALLEY|        LANCASHIRE|
|         {0000A30F-83DE-4C...|     94000|2005-05-25 00:00:00|            S|      N|       F|        MANSFIELD|           MANSFIELD|   NOTTINGHAMSHIRE|
|         {00013EEA-4E77-4C...|    104000|2007-08-31 00:00:00|            S|      N|    

Теперь проверим наши данные еще раз.

In [ ]:
string_columns1 = ["Transaction_unique_identifier",  "Property_Type", "Old/New",
                  "Duration", "Town/City", "District", "County"]
numeric_columns1 = ["Price Paid"]
date_columms1 =  ["Date_of_Transfer"]
missing_values1 = {}
for index, column in enumerate(df_updated_1.columns):
    if column in string_columns1:    # проверить столбцы строк со значениями None и Null
        missing_count1 = df_updated_1.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
        missing_values1.update({column:missing_count1})
    if column in numeric_columns1:  # check zeroes, None, NaN
        missing_count1 = df_updated_1.where(col(column).isin([0,None,np.nan])).count()
        missing_values1.update({column:missing_count1})
    if column in date_columms1:  # check zeroes, None, NaN
        missing_count1 = df_updated_1.where(col(column).isNull()).count()
        missing_values1.update({column: missing_count1})
missing_df1 = pd.DataFrame.from_dict([missing_values1])
missing_df1

,Transaction_unique_identifier,Price Paid,Date_of_Transfer,Property_Type,Old/New,Duration,Town/City,District,County
0,0,0,0,0,0,0,0,0,0


Теперь данные очищены.

# **4. Исследовательский анализ данных** <a class="anchor" id="5"></a>

Расчет средней цены по месяцам

In [ ]:
from pyspark.sql.functions import udf, col, avg, to_timestamp

# Вспомогательная функция для извлечения года и месяца из даты
def extract_year_month(date_obj):
    if date_obj:  # Проверяем на пустую строку
        return f"{date_obj.year}-{date_obj.month:02d}"
    return None

# Регистрация UDF
udf_extract_year_month = udf(extract_year_month, StringType())

In [ ]:
# Создание нового столбца year_month с помощью UDF
df_updated_1 = df_updated_1.withColumn("year_month", udf_extract_year_month(col("Date_of_Transfer")))

# Группируем по year_month и вычисляем среднюю цену
result = df_updated_1.groupBy("year_month").agg(avg("Price Paid").alias("avg_price"))

# Сортируем по year_month
result = result.orderBy("year_month")

# Проверка результата
result.show()

+----------+-----------------+
|year_month|        avg_price|
+----------+-----------------+
|   1995-01|68382.62340775372|
|   1995-02|65650.63058013766|
|   1995-03|65751.50024547966|
|   1995-04|67833.55898745936|
|   1995-05|67084.04479129819|
|   1995-06|67988.24961008578|
|   1995-07|70311.59702481773|
|   1995-08|70604.03303225455|
|   1995-09|68369.13111452943|
|   1995-10|67571.96740567633|
|   1995-11|67059.57860801979|
|   1995-12|68024.27600609507|
|   1996-01|68255.07566926992|
|   1996-02| 66529.8499531964|
|   1996-03| 66266.8594251821|
|   1996-04|69695.42989435433|
|   1996-05|69471.80959102382|
|   1996-06|71028.67403971191|
|   1996-07|73714.84411999468|
|   1996-08|74734.46267280151|
+----------+-----------------+
only showing top 20 rows



Теперь это надо провизуализировать для наглядности.

In [ ]:
import pandas as pd
import altair as alt

# Преобразуем результат в Pandas DataFrame
df_updated_2 = result.toPandas()

# Проверка структуры данных
print(df_updated_2.head())

# Создаем линейный график
alt.Chart(df_updated_2).mark_line().encode(x='year_month', y='avg_price', tooltip=['year_month', 'avg_price']).properties(title='Average Price Paid Over Time').interactive()

  year_month     avg_price
0    1995-01  68382.623408
1    1995-02  65650.630580
2    1995-03  65751.500245
3    1995-04  67833.558987
4    1995-05  67084.044791


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Наблюдается положительный тренд в росте средней цены на недвижимость, при этом с 2020 года цены стали меняться более резко. В целом это можно объяснить несколькими причинами:
1. Это был период пандемии, все сидели дома и те, кто жил не один, могли бы устать от соседей по комнате и больше интересоваться поисками жилья.
2.  В июле 2020 года правительство ввело приостановку гербового сбора — налога на покупку домов.
3. При этом в 2021 году отменили льготы, поэтому спрос на жилье ниже, чем в 2020.
4. Цены с каждым годом меняются, как и зараплаты, курс доллара и тп.

Такой же график можно рассмотреть с фильтрацией по городам, но уже берем интервал за последние 10 лет.

In [13]:
import panel as pn
import altair as alt
from pyspark.sql.functions import year, avg

# Загрузить расширение Panel
pn.extension()

<ipython-input-13-4c764bca30e5>:6: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


In [14]:
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [15]:
pn.extension('vega')

In [16]:
# Фильтруем данные за последние 10 лет
current_year_value = 2023
last_10_years_df = df_updated_1.filter(year("Date_of_Transfer") >= (current_year_value - 10))

# Добавляем столбец Year, используя year() для группировки
last_10_years_df = last_10_years_df.withColumn("Year", year("Date_of_Transfer"))

# Группировка данных для получения средней цены по дате и типу недвижимости
average_price_df = last_10_years_df.groupBy("Year", "Town/City").agg(avg("Price Paid").alias("Avg_Price"))

# Преобразование в Pandas DataFrame
pdf = average_price_df.toPandas()

print(pdf.head(2))

   Year  Town/City      Avg_Price
0  2015  WORCESTER  248511.394309
1  2014  BRENTFORD  421321.577273


In [17]:
# Функция для создания интерактивного графика
def create_chart(city_type=None):
    print("Initial data points:", len(pdf))

    filtered_data = pdf

    # Если выбран 'All', то агрегируем среднюю цену по всем городам
    if city_type == 'All':
        filtered_data = pdf.groupby('Year', as_index=False).agg({'Avg_Price': 'mean'}).copy()
        filtered_data['Town/City'] = 'All Cities'  # Указываем, что это общее значение
    elif city_type:
        filtered_data = filtered_data[filtered_data['Town/City'] == city_type]

    print("Filtered data points for {}: {}".format(city_type, len(filtered_data)))

    # Построение графика
    chart = alt.Chart(filtered_data).mark_line().encode(
        x='Year:O',  # Убедитесь, что Year является категорийным для корректного отображения
        y='Avg_Price:Q',
        color='Town/City:N',
        tooltip=['Year', 'Avg_Price', 'Town/City']
    ).properties(
        title='Average Price Paid Year by Town/City'
    )
    return chart

In [18]:
sorted_data = average_price_df.orderBy(col('Avg_Price').desc())

# Выводим результат
sorted_data.select('Town/City', 'Avg_Price').show()

+--------------------+------------------+
|           Town/City|         Avg_Price|
+--------------------+------------------+
|             GATWICK|             1.4E8|
|             GATWICK|           4.075E7|
|             GATWICK|           2.075E7|
|             GATWICK|       2.0346875E7|
|             GATWICK|          1.3525E7|
|             GATWICK|         4000000.0|
|LLANFAIRPWLLGWYNGYLL|2711658.4615384615|
|      VIRGINIA WATER|2590052.6315789474|
|             STAINES|2430333.3333333335|
|      VIRGINIA WATER| 2322465.068627451|
|                IVER| 2245508.487654321|
|          WINDLESHAM|2119913.8157894737|
|             GATWICK|         2024994.5|
|      VIRGINIA WATER| 1967638.954954955|
|      VIRGINIA WATER|1877294.4942528736|
|      VIRGINIA WATER|1802306.4102564103|
|              KESTON|         1780000.0|
|               ESHER|1515742.5395480227|
|          WINCHELSEA|1467666.6666666667|
|              COBHAM|1459730.6565656567|
+--------------------+------------

In [20]:
# Создаем выпадающий список для выбора городов, возьмем 5 самых дорогих по недвижимости
city_types = ['GATWICK', 'LLANFAIRPWLLGWYNGYLL', 'VIRGINIA WATER', 'STAINES', 'IVER']
city_selector = pn.widgets.Select(name='Select Town/City', options=['All'] + city_types)

# Создаем интерактивный дашборд
@pn.depends(city_selector.param.value)
def update_dashboard(city_type):
    chart = create_chart(city_type)  # Возвращаем данные в зависимости от выбранного типа
    return chart

# Создаем панель дашборда
dashboard = pn.Column(
    city_selector,
    update_dashboard
)

# Запускаем дашборд
dashboard.servable()

Initial data points: 12607
Filtered data points for All: 11


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


Initial data points: 12607 
Filtered data points for VIRGINIA WATER: 11 
Initial data points: 12607 
Filtered data points for STAINES: 11 
Initial data points: 12607 
Filtered data points for IVER: 11

Column
    [0] Select(options=['All', 'GATWICK', ...], value='All')
    [1] ParamFunction(function, _pane=Vega, defer_load=False)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: Futu

Такие же графики можно построить и для следующих столбцов Property_Type, Old/New, County.

In [21]:
unique_values = last_10_years_df.select("Property_Type").distinct().collect()

# Вывод уникальных значений
for row in unique_values:
    print(row["Property_Type"])

F
T
D
O
S


In [22]:
# Группировка данных для получения средней цены по дате и типу недвижимости
average_price_df_1 = last_10_years_df.groupBy("Year", "Property_Type").agg(avg("Price Paid").alias("Avg_Price_1"))

# Преобразование в Pandas DataFrame
pdf_1 = average_price_df_1.toPandas()

# Функция для создания интерактивного графика
def create_chart(property_type=None):
    print("Initial data points:", len(pdf_1))

    filtered_data_1 = pdf_1

    # Если выбран 'All', то агрегируем среднюю цену по всем типам
    if property_type == 'All':
        filtered_data_1 = pdf_1.groupby('Year', as_index=False).agg({'Avg_Price_1': 'mean'}).copy()
        filtered_data_1['Property_Type'] = 'All Property_Type'  # Указываем, что это общее значение
    elif property_type:
        filtered_data_1 = filtered_data_1[filtered_data_1['Property_Type'] == property_type]

    print("Filtered data points for {}: {}".format(property_type, len(filtered_data_1)))

    # Построение графика
    chart = alt.Chart(filtered_data_1).mark_line().encode(
        x='Year:O',  # Убедитесь, что Year является категорийным для корректного отображения
        y='Avg_Price_1:Q',
        color='Property_Type:N',
        tooltip=['Year', 'Avg_Price_1', 'Property_Type']
    ).properties(
        title='Average Price Paid Year by Property_Type'
    )
    return chart

# Создаем выпадающий список для выбора
property_types = ['F', 'T', 'D', 'O', 'S']
property_selector = pn.widgets.Select(name='Select Property_Type', options=['All'] + property_types)

# Создаем интерактивный дашборд
@pn.depends(property_selector.param.value)
def update_dashboard(property_type):
    chart = create_chart(property_type)  # Возвращаем данные в зависимости от выбранного типа
    return chart

# Создаем панель дашборда
dashboard = pn.Column(
    property_selector,
    update_dashboard
)

# Запускаем дашборд
dashboard.servable()

Initial data points: 55
Filtered data points for All: 11


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


Initial data points: 55 
Filtered data points for F: 11 
Initial data points: 55 
Filtered data points for T: 11 
Initial data points: 55 
Filtered data points for D: 11 
Initial data points: 55 
Filtered data points for O: 11 
Initial data points: 55 
Filtered data points for S: 11

Column
    [0] Select(options=['All', 'F', 'T', ...], value='All')
    [1] ParamFunction(function, _pane=Vega, defer_load=False)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: Futu

In [23]:
# Группировка данных для получения средней цены по дате и типу недвижимости
average_price_df_2 = last_10_years_df.groupBy("Year", "Old/New").agg(avg("Price Paid").alias("Avg_Price_2"))

# Преобразование в Pandas DataFrame
pdf_2 = average_price_df_2.toPandas()

# Функция для создания интерактивного графика
def create_chart(oldnew_type=None):
    print("Initial data points:", len(pdf_2))

    filtered_data_2 = pdf_2

    # Если выбран 'All', то агрегируем среднюю цену по всем типам
    if oldnew_type == 'All':
        filtered_data_2 = pdf_2.groupby('Year', as_index=False).agg({'Avg_Price_2': 'mean'}).copy()
        filtered_data_2['Old/New'] = 'All Old/New'  # Указываем, что это общее значение
    elif oldnew_type:
        filtered_data_2 = filtered_data_2[filtered_data_2['Old/New'] == oldnew_type]

    print("Filtered data points for {}: {}".format(oldnew_type, len(filtered_data_2)))

    # Построение графика
    chart = alt.Chart(filtered_data_2).mark_line().encode(
        x='Year:O',  # Убедитесь, что Year является категорийным для корректного отображения
        y='Avg_Price_2:Q',
        color='Old/New:N',
        tooltip=['Year', 'Avg_Price_2', 'Old/New']
    ).properties(
        title='Average Price Paid Year by Old/New'
    )
    return chart

# Создаем выпадающий список для выбора
oldnew_types = ['Y', 'N']
oldnew_selector = pn.widgets.Select(name='Select Old/New', options=['All'] + oldnew_types)

# Создаем интерактивный дашборд
@pn.depends(oldnew_selector.param.value)
def update_dashboard(oldnew_type):
    chart = create_chart(oldnew_type)  # Возвращаем данные в зависимости от выбранного типа
    return chart

# Создаем панель дашборда
dashboard = pn.Column(
    oldnew_selector,
    update_dashboard
)

# Запускаем дашборд
dashboard.servable()

Initial data points: 22
Filtered data points for All: 11


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


Initial data points: 22 
Filtered data points for Y: 11 
Initial data points: 22 
Filtered data points for N: 11

Column
    [0] Select(options=['All', 'Y', 'N'], value='All')
    [1] ParamFunction(function, _pane=Vega, defer_load=False)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: Futu

In [ ]:
unique_values = last_10_years_df.select("County").distinct().collect()

# Вывод уникальных значений
for row in unique_values:
    print(row["County"])

CUMBERLAND
GREATER MANCHESTER
RUTLAND
LEICESTERSHIRE
OXFORDSHIRE
MILTON KEYNES
MERTHYR TYDFIL
NORTHAMPTONSHIRE
MONMOUTHSHIRE
BRIGHTON AND HOVE
THE VALE OF GLAMORGAN
WREKIN
WOKINGHAM
SOUTH YORKSHIRE
SLOUGH
WEST NORTHAMPTONSHIRE
SOUTH GLOUCESTERSHIRE
YORK
DERBYSHIRE
HEREFORDSHIRE
EAST SUSSEX
EAST RIDING OF YORKSHIRE
WEST YORKSHIRE
LEICESTER
CHESHIRE EAST
REDCAR AND CLEVELAND
TORFAEN
CITY OF KINGSTON UPON HULL
CONWY
CITY OF NOTTINGHAM
CUMBRIA
STOKE-ON-TRENT
HALTON
NEATH PORT TALBOT
WREXHAM
GWYNEDD
WEST SUSSEX
BRIDGEND
DARLINGTON
BRACKNELL FOREST
CAMBRIDGESHIRE
THURROCK
WEST MIDLANDS
WORCESTERSHIRE
CAERPHILLY
MERSEYSIDE
WEST BERKSHIRE
POOLE
STOCKTON-ON-TEES
NOTTINGHAMSHIRE
KENT
NORTH LINCOLNSHIRE
SOMERSET
NORTHUMBERLAND
BLAENAU GWENT
DENBIGHSHIRE
PEMBROKESHIRE
SWANSEA
BUCKINGHAMSHIRE
CITY OF PETERBOROUGH
ISLE OF WIGHT
MIDDLESBROUGH
TORBAY
ESSEX
BOURNEMOUTH, CHRISTCHURCH AND POOLE
FLINTSHIRE
PORTSMOUTH
CITY OF PLYMOUTH
BATH AND NORTH EAST SOMERSET
WILTSHIRE
CARDIFF
CITY OF BRISTOL
NORTH NOR

In [24]:
# Группировка данных для получения средней цены по дате и типу недвижимости
average_price_df_3 = last_10_years_df.groupBy("Year", "County").agg(avg("Price Paid").alias("Avg_Price_3"))

# Преобразование в Pandas DataFrame
pdf_3 = average_price_df_3.toPandas()

# Функция для создания интерактивного графика
def create_chart(country_type=None):
    print("Initial data points:", len(pdf_3))

    filtered_data_3 = pdf_3

    # Если выбран 'All', то агрегируем среднюю цену по всем городам
    if country_type == 'All':
        filtered_data_3 = pdf_3.groupby('Year', as_index=False).agg({'Avg_Price_3': 'mean'}).copy()
        filtered_data_3['County'] = 'All County'  # Указываем, что это общее значение
    elif country_type:
        filtered_data_3 = filtered_data_3[filtered_data_3['County'] == country_type]

    print("Filtered data points for {}: {}".format(country_type, len(filtered_data_3)))

    # Построение графика
    chart = alt.Chart(filtered_data_3).mark_line().encode(
        x='Year:O',  # Убедитесь, что Year является категорийным для корректного отображения
        y='Avg_Price_3:Q',
        color='County:N',
        tooltip=['Year', 'Avg_Price_3', 'County']
    ).properties(
        title='Average Price Paid Year by County'
    )
    return chart

# Создаем выпадающий список для выбора городов, возьмем 5 самых крупных
country_types = ['CUMBERLAND', 'GREATER MANCHESTER', 'RUTLAND', 'LEICESTERSHIRE', 'OXFORDSHIRE']
country_selector = pn.widgets.Select(name='Select County', options=['All'] + country_types)

# Создаем интерактивный дашборд
@pn.depends(country_selector.param.value)
def update_dashboard(country_type):
    chart = create_chart(country_type)  # Возвращаем данные в зависимости от выбранного типа
    return chart

# Создаем панель дашборда
dashboard = pn.Column(
    country_selector,
    update_dashboard
)

# Запускаем дашборд
dashboard.servable()

Initial data points: 1268
Filtered data points for All: 11


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


Initial data points: 1268 
Filtered data points for CUMBERLAND: 6 
Initial data points: 1268 
Filtered data points for GREATER MANCHESTER: 11 
Initial data points: 1268 
Filtered data points for RUTLAND: 11 
Initial data points: 1268 
Filtered data points for GREATER MANCHESTER: 11 
Initial data points: 1268 
Filtered data points for RUTLAND: 11 
Initial data points: 1268 
Filtered data points for LEICESTERSHIRE: 11 
Initial data points: 1268 
Filtered data points for OXFORDSHIRE: 11

Column
    [0] Select(options=['All', 'CUMBERLAND', ...], value='All')
    [1] ParamFunction(function, _pane=Vega, defer_load=False)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: Futu

Статистика для старых и новых объектов

In [ ]:
from pyspark.sql import functions as F

avg_price_by_old_new = df_updated_1.groupBy("Old/New").agg(
    F.avg("Price Paid").alias("Avg_Price"),
    F.min("Price Paid").alias("Min_Price"),
    F.max("Price Paid").alias("Max_Price"),
    F.stddev("Price Paid").alias("StdDev_Price")
)

avg_price_by_old_new.show()

+-------+------------------+---------+---------+------------------+
|Old/New|         Avg_Price|Min_Price|Max_Price|      StdDev_Price|
+-------+------------------+---------+---------+------------------+
|      Y|220452.25080930127|      100|110900000|313574.65510593896|
|      N|217864.30495400575|        1|594300000| 911141.5615617076|
+-------+------------------+---------+---------+------------------+



Новые дома стоят чуть дешевле старых по средней цене, при этом минимальная цена нового дома меньше, чем старого. Это говорит о том, что цены на новую недвижимость более вариативны, то есть у них больше разброс, чем у старой недвижимости.

Статистика по округам

In [ ]:
from pyspark.sql import functions as F

avg_price_by_district = df_updated_1.groupBy("District").agg(
    F.avg("Price Paid").alias("Avg_Price"),
    F.min("Price Paid").alias("Min_Price"),
    F.max("Price Paid").alias("Max_Price"),
    F.stddev("Price Paid").alias("StdDev_Price")
)

avg_price_by_district.show()

+------------------+------------------+---------+---------+------------------+
|          District|         Avg_Price|Min_Price|Max_Price|      StdDev_Price|
+------------------+------------------+---------+---------+------------------+
|      NORTH DORSET| 198326.4233847412|      500|  7125000|165678.91591807327|
|BERWICK-UPON-TWEED|106986.04163644277|     8000|  1680000| 85971.71819998963|
|CREWE AND NANTWICH|113353.26357900775|     8000|  1525000|  91026.9907967185|
|        CUMBERLAND| 371805.6844444444|    22000|  3474000| 736915.5298918247|
|         ROTHERHAM|121564.44493178297|      100| 89450000| 355692.7585993366|
|         ABERCONWY|  53344.0939675174|     5500|   239000|28975.888163615804|
|           CHORLEY|150715.78273231623|      115| 37740000|233826.65822307832|
|          SANDWELL| 118401.9851073508|      300|167499800| 545158.1036010637|
|           RUTLAND|  240078.989583838|      500|  5660000| 221715.2010633359|
|           FAREHAM|219850.96151652923|      100| 52

Статистика по городам

In [ ]:
from pyspark.sql import functions as F

avg_price_by_city = df_updated_1.groupBy("Town/City").agg(
    F.avg("Price Paid").alias("Avg_Price"),
    F.min("Price Paid").alias("Min_Price"),
    F.max("Price Paid").alias("Max_Price"),
    F.stddev("Price Paid").alias("StdDev_Price")
)

avg_price_by_city.show()

+-------------------+------------------+---------+---------+------------------+
|          Town/City|         Avg_Price|Min_Price|Max_Price|      StdDev_Price|
+-------------------+------------------+---------+---------+------------------+
| BERWICK-UPON-TWEED|135281.20986411677|      500|  5095000|130164.86485728573|
|       HIGH WYCOMBE| 269439.8232519871|      100| 38750000|470449.41467125196|
|ST. LEONARDS-ON-SEA|101101.18453745668|     5250|   975000| 70368.43143288426|
|            NEWQUAY|211518.58508100588|      100| 10259480|228542.67709051704|
|           GOODWICK|127167.76673040153|     2000|   975000|102841.46311813417|
|          ROTHERHAM|114841.08626248685|      100| 89450000| 362354.5167024273|
|          LLANGEFNI| 142408.0246863618|      500|  2430000|125252.13213936634|
|           STAMFORD|224342.58489688323|      100|  9000000|231018.37882194898|
|  LEE-ON-THE-SOLENT|218025.98260192157|     3184|  1900459|133515.46207130258|
|        CHOPPINGTON| 85640.77870757788|

Статистика по странам

In [ ]:
from pyspark.sql import functions as F

avg_price_by_country = df_updated_1.groupBy("County").agg(
    F.avg("Price Paid").alias("Avg_Price"),
    F.min("Price Paid").alias("Min_Price"),
    F.max("Price Paid").alias("Max_Price"),
    F.stddev("Price Paid").alias("StdDev_Price")
)

avg_price_by_country.show()

+--------------------+------------------+---------+---------+------------------+
|              County|         Avg_Price|Min_Price|Max_Price|      StdDev_Price|
+--------------------+------------------+---------+---------+------------------+
|          CUMBERLAND| 371805.6844444444|    22000|  3474000| 736915.5298918247|
|  GREATER MANCHESTER| 148148.6654620539|        1|292000000| 599933.3868881704|
|      LEICESTERSHIRE| 186417.9396913738|      100|135087821|  663918.030312774|
|             RUTLAND|  240078.989583838|      500|  5660000|221715.20106333584|
|         OXFORDSHIRE| 294791.3279481517|      100| 93395000| 743762.8907341509|
|       MILTON KEYNES|210962.38845198674|      200|146750000| 812515.0685290957|
|      MERTHYR TYDFIL| 92645.03383268035|      100| 14860000|173922.64991182173|
|    NORTHAMPTONSHIRE|164581.73067717394|      100| 77775000| 457642.8569777278|
|                AVON| 64513.38536971617|     7500|   940000| 39427.33533684403|
|       MONMOUTHSHIRE| 21331

Поделить транзакции на категории высоких, средних и низких цен.

In [ ]:
from pyspark.sql.functions import when, percentile_approx
approx_25 = df_updated_1.agg(percentile_approx(col("Price Paid"), 0.25).alias("approx_25_percentile")).collect()[0]["approx_25_percentile"]
approx_75 = df_updated_1.agg(percentile_approx(col("Price Paid"), 0.75).alias("approx_75_percentile")).collect()[0]["approx_75_percentile"]

In [ ]:
df_precent = df_updated_1.withColumn("approx_25_percentile", when(col("Price Paid").isNotNull(), approx_25)) \
                          .withColumn("approx_75_percentile", when(col("Price Paid").isNotNull(), approx_75))

In [ ]:
df_precent = df_precent.withColumn("Price_Category",
                   when(col("Price Paid") < col("approx_25_percentile"), "Low")
                   .when(col("Price Paid") < col("approx_75_percentile"), "Medium")
                   .otherwise("High")
                   )

# Группируем по категориям и считаем количество
df_precent.groupBy("Price_Category").count().show()

+--------------+--------+
|Price_Category|   count|
+--------------+--------+
|          High| 7072890|
|           Low| 7055266|
|        Medium|14148072|
+--------------+--------+

